Trying to aggregate data in the minutes table and adding it to the hours table

Creating an Hours table

In [1]:
import boto3

dynamodb = boto3.resource('dynamodb')
table = dynamodb.create_table(
    TableName='trial-blackout-monitor-hours',
    KeySchema=[
        {
            'AttributeName': 'pk',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'sk',
            'KeyType': 'RANGE'
        }
    ],
    AttributeDefinitions=[
        {
            "AttributeName": "pk",
            "AttributeType": "S"
        },
        {
            "AttributeName": "sk",
            "AttributeType": "S"
        }
    ],
    ProvisionedThroughput = {
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    }
)

Code that reads off values and aggregates them 

In [ ]:
import boto3
import platform
import datetime
from boto3.dynamodb.conditions import Key

READ_PIN = 4
dynamodb = boto3.resource('dynamodb')
mins_table = dynamodb.Table('dev-blackout-monitor-minutes')
hours_table = dynamodb.Table('')

results = mins_table.query(
    IndexName = "sk_status-index",
    KeyConditionExpression=Key("pk_status").eq("device#alex_rpi#sensor#4#status#new")
)
results["Items"]
while len(results["Items"]) > 0:
    # getting the data about an hour
    first_result = results["Items"][0]
    by_hour_readings = table.query(KeyConditionExpression=Key("pk").eq(first_result["pk"]))
    on_r, off_r, unk_r= 0, 0, 60
    for each_reading in by_hour_readings["Items"]:
        on_r += 1 if each_reading["reading"] == 1 else 0
        off_r += 1 if each_reading["reading"] == 0 else 0
    count = on_r + off_r    
    unk_r = 60 - count
    print(first_result["pk"], on_r, off_r, count, unk_r)
    # adding data about an hour to the table
    hours_table.put_item(
        Item={
            "pk": f"{first_result['pk'].rsplit('-', 1)[0]}",
            "sk": f"{first_result['pk'].rsplit('-', 1)[1]}",
            "count": count,
            "expected": 60,
            "off": off_r,
            "on": on_r,
            "pk_status": f"device#{platform.node()}#sensor{READ_PIN}#status#new",
            "ts": f"{datetime.datetime.now(datetime.timezone.utc).isoformat()}",
            "unknown": unk_r

        }
    )
    # making a list of minutes that need to be updated
    mnts_to_upd = list()
    for reading in results["Items"]:
        if reading["pk"] == first_result["pk"]:
            mnts_to_upd.append(int(reading["sk"].split("#")))
    # changing statuses of the entries that were accounted for
    for minute in mnts_to_upd:
        update = mins_table.update_item(
            Key={
                'pk': first_result["pk"],
                'sk': f"time#{minute}"
            },
            UpdateExpression="set pk_status :s",
            ExpressionAttributeValues={
                ':s': f"device#{platform.node()}#sensor{READ_PIN}#status#done"
            }
        )
    # querying data again
    results = mins_table.query(
        IndexName = "sk_status-index",
        KeyConditionExpression=Key("pk_status").eq("device#alex_rpi#sensor#4#status#new")
    )
# first_result = results["Items"][0]
# by_hour_readings = mins_table.query(KeyConditionExpression=Key("pk").eq(first_result["pk"]))
# on_r, off_r, unk_r= 0, 0, 60
# for each_reading in by_hour_readings["Items"]:
#     on_r += 1 if each_reading["reading"] == 1 else 0
#     off_r += 1 if each_reading["reading"] == 0 else 0
# count = on_r + off_r    
# unk_r = 60 - count
# print(first_result["pk"], on_r, off_r, count, unk_r)  
# print(first_result["pk"].rsplit('-', 1)[0])
# print(first_result["pk"].rsplit('-', 1)[1])
# print(results["Items"][0])

SyntaxError: f-string: unmatched '[' (3522261276.py, line 30)